In [1]:
#Imports
import pandas as pd
import os
from google.cloud import bigquery
from datetime import datetime, date, timedelta
import re
import calendar
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from PIL import Image

pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None  # default='warn'

from pathlib import Path 

In [2]:
group = 'Restaurants'
#TMT+
#Travel/Leisure
#Softlines
#Grocery/Discount/Club
#Hardlines
#Restaurants
#Financial Services

now = datetime.now()
day = now.strftime("%d")
month = now.strftime("%m")
year = now.strftime("%Y")
mm_dd_yyyy = month+'_'+day+'_'+year

first_trans_date = '2024-06-01'

In [3]:
#make new folder for today's run
#root = 'C:/Users/Greg/AppData/Local/Programs/Python/Python39/Output/anomaly_detection/'+group+'/'
#date_path = month+'_'+day+'_'+year+'/'
#directory = Path(root+date_path)
#mode = 0o666

#if not os.path.exists(directory):
#    os.mkdir(directory)


In [4]:
#Bigquery credentials/Date transformations
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=r"C:\Users\Greg\OneDrive - Consumer Edge Research LLC\Desktop\Python\gcp-key-2.json"
#C:\Users\Greg\AppData\Local\Programs\Python\Python39\gcp-key-2.json
#C:\Users\Greg\OneDrive - Consumer Edge Research LLC\Desktop\Python\gcp-key-2.json
client = bigquery.Client()

quarters = {
  1: "Q1",
  2: "Q1",
  3: "Q1",
  4: "Q2",
  5: "Q2",
  6: "Q2",
  7: "Q3",
  8: "Q3",
  9: "Q3",
  10: "Q4",
  11: "Q4",
  12: "Q4",
}

#Trans_date transformations
def date_transf(df):
    df['trans_date_dt'] = pd.to_datetime(df['trans_date'], format = "%Y-%m-%d", errors = 'coerce')
    df['year'] = df['trans_date_dt'].dt.year.astype(str)
    df['month_dt'] = df['trans_date_dt'].dt.month
    df['month'] = df['month_dt'].apply(lambda x: calendar.month_abbr[x])
    df['day'] = df['trans_date_dt'].dt.day
    df['month_day'] = df['trans_date_dt'].dt.strftime("%b-%d")
    df['norm_date'] = df['trans_date_dt'].dt.strftime("%m/%d/%Y")
    df['period'] = df['trans_date_dt'].dt.to_period('Q')
    df['norm_date_ya'] = df['norm_date'].shift(364)
    


In [5]:
#Fetch USA1 and USA1+USA2 segment data
#us1_day_seg_query = 'SELECT *, CASE WHEN industry_name = "Financial Services" THEN "Financial Services"  WHEN industry_name = "Insurance" THEN "Financial Services" WHEN industry_name = "Convenience/Drug/Diet" THEN "Grocery/Discount/Club" WHEN industry_name = "Grocery" THEN "Grocery/Discount/Club" WHEN industry_name = "Health Products/Services" THEN "Grocery/Discount/Club" WHEN industry_name = "Discount/Club" THEN "Grocery/Discount/Club" WHEN industry_name = "Home & Garden" THEN "Hardlines" WHEN industry_name = "Automotive" THEN "Hardlines" WHEN industry_name = "Beauty" THEN "Hardlines" WHEN industry_name = "Sporting Goods, Hobby, Toy & Game" THEN "Hardlines" WHEN industry_name = "Commercial - Hardlines" THEN "Hardlines" WHEN industry_name = "Pet Products/Services" THEN "Hardlines" WHEN industry_name = "Limited-Service Restaurants" THEN "Restaurants" WHEN industry_name = "Full-Service Restaurants" THEN "Restaurants" WHEN industry_name = "Apparel, Accessories & Footwear" THEN "Softlines" WHEN industry_name = "Luxury" THEN "Softlines" WHEN industry_name = "Department Stores" THEN "Softlines" WHEN industry_name = "Electronics/Software" THEN "TMT+" WHEN industry_name = "Miscellaneous Services" THEN "TMT+" WHEN industry_name = "Online Retail - Broadlines" THEN "TMT+" WHEN industry_name = "Telecom & Media" THEN "TMT+" WHEN industry_name = "Education" THEN "TMT+" WHEN industry_name = "Other Specialty Retail" THEN "TMT+" WHEN industry_name = "Leisure & Recreation" THEN "Travel/Leisure" WHEN industry_name = "Travel" THEN "Travel/Leisure" WHEN industry_name = "Ground Transportation" THEN "Travel/Leisure" ELSE "" END as Insights_Group FROM(SELECT *, LAG(rolling_7d_spend_growth,364) over(order by symbol, segment_name, trans_date) as rolling_7d_spend_growth_ya, LAG(rolling_35d_spend_growth,364) over(order by symbol, segment_name, trans_date) as rolling_35d_spend_growth_ya FROM (SELECT *, ROUND((rolling_7d_spend/LAG(rolling_7d_spend,364) over(order by symbol, segment_name, trans_date)-1),1) as rolling_7d_spend_growth, ROUND((rolling_35d_spend/LAG(rolling_35d_spend,364) over(order by symbol, segment_name, trans_date, trans_date)-1),1) as rolling_35d_spend_growth FROM (SELECT t.symbol, t.segment_name, t.trans_date, t.trans_count, t.spend_amount, industry_name, ind.subindustry_name, SUM(spend_amount) over(order by t.symbol, t.segment_name, t.trans_date ROWS BETWEEN 6 PRECEDING and CURRENT Row) as rolling_7d_spend, SUM(t.spend_amount) over(order by t.symbol, t.segment_name, t.trans_date ROWS BETWEEN 34 PRECEDING and CURRENT Row) as rolling_35d_spend FROM `beta-omega-alpha-theta.apollo_us_cube_daily_signal_legacy.day_sym_seg_emax` t INNER JOIN `beta-omega-alpha-theta.ground_truth.symbol_segment` sym_seg ON t.segment_name = sym_seg.segment_name INNER JOIN `beta-omega-alpha-theta.ground_truth.symbol_detail` sym_det ON sym_seg.symbol_id = sym_det.symbol_id INNER JOIN `beta-omega-alpha-theta.ground_truth.industry_list` ind ON sym_det.subindustry_id = ind.subindustry_id order by symbol, segment_name, trans_date))) order by symbol, segment_name, trans_date DESC'
#us1_day_seg_df = client.query(us1_day_seg_query).to_dataframe()
#us1_day_seg_df.sort_values(by = ['symbol', 'segment_name', 'trans_date'], inplace = True)

us1_2_day_seg_query = 'SELECT *, CASE WHEN industry_name = "Financial Services" THEN "Financial Services"  WHEN industry_name = "Insurance" THEN "Financial Services" WHEN industry_name = "Convenience/Drug/Diet" THEN "Grocery/Discount/Club" WHEN industry_name = "Grocery" THEN "Grocery/Discount/Club" WHEN industry_name = "Health Products/Services" THEN "Grocery/Discount/Club" WHEN industry_name = "Discount/Club" THEN "Grocery/Discount/Club" WHEN industry_name = "Home & Garden" THEN "Hardlines" WHEN industry_name = "Automotive" THEN "Hardlines" WHEN industry_name = "Beauty" THEN "Hardlines" WHEN industry_name = "Sporting Goods, Hobby, Toy & Game" THEN "Hardlines" WHEN industry_name = "Commercial - Hardlines" THEN "Hardlines" WHEN industry_name = "Pet Products/Services" THEN "Hardlines" WHEN industry_name = "Limited-Service Restaurants" THEN "Restaurants" WHEN industry_name = "Full-Service Restaurants" THEN "Restaurants" WHEN industry_name = "Apparel, Accessories & Footwear" THEN "Softlines" WHEN industry_name = "Luxury" THEN "Softlines" WHEN industry_name = "Department Stores" THEN "Softlines" WHEN industry_name = "Electronics/Software" THEN "TMT+" WHEN industry_name = "Miscellaneous Services" THEN "TMT+" WHEN industry_name = "Online Retail - Broadlines" THEN "TMT+" WHEN industry_name = "Telecom & Media" THEN "TMT+" WHEN industry_name = "Education" THEN "TMT+" WHEN industry_name = "Other Specialty Retail" THEN "TMT+" WHEN industry_name = "Leisure & Recreation" THEN "Travel/Leisure" WHEN industry_name = "Travel" THEN "Travel/Leisure" WHEN industry_name = "Ground Transportation" THEN "Travel/Leisure" ELSE "" END as Insights_Group FROM(SELECT *, LAG(rolling_7d_spend_growth,364) over(order by symbol, segment_name, trans_date) as rolling_7d_spend_growth_ya, LAG(rolling_35d_spend_growth,364) over(order by symbol, segment_name, trans_date) as rolling_35d_spend_growth_ya FROM (SELECT *, ROUND((rolling_7d_spend/LAG(rolling_7d_spend,364) over(order by symbol, segment_name, trans_date)-1),3) as rolling_7d_spend_growth, ROUND((rolling_35d_spend/LAG(rolling_35d_spend,364) over(order by symbol, segment_name, trans_date, trans_date)-1),3) as rolling_35d_spend_growth FROM (SELECT t.symbol, t.segment_name, t.trans_date, t.trans_count, t.spend_amount, industry_name, ind.subindustry_name, SUM(spend_amount) over(order by t.symbol, t.segment_name, t.trans_date ROWS BETWEEN 6 PRECEDING and CURRENT Row) as rolling_7d_spend, SUM(t.spend_amount) over(order by t.symbol, t.segment_name, t.trans_date ROWS BETWEEN 34 PRECEDING and CURRENT Row) as rolling_35d_spend FROM `beta-omega-alpha-theta.combined_us1_us2_cube_daily_signal_legacy.day_sym_seg_emax` t INNER JOIN `beta-omega-alpha-theta.ground_truth.symbol_segment` sym_seg ON t.segment_name = sym_seg.segment_name INNER JOIN `beta-omega-alpha-theta.ground_truth.symbol_detail` sym_det ON sym_seg.symbol_id = sym_det.symbol_id INNER JOIN `beta-omega-alpha-theta.ground_truth.industry_list` ind ON sym_det.subindustry_id = ind.subindustry_id order by symbol, segment_name, trans_date))) order by symbol, segment_name, trans_date DESC'
us1_2_day_seg_df = client.query(us1_2_day_seg_query).to_dataframe()
us1_2_day_seg_df.sort_values(by = ['symbol', 'segment_name', 'trans_date'], inplace = True)

#Fetch USA1 and USA1+USA2 brand data
#us1_day_brand_query = 'SELECT *, CASE WHEN industry_name = "Financial Services" THEN "Financial Services"  WHEN industry_name = "Insurance" THEN "Financial Services" WHEN industry_name = "Convenience/Drug/Diet" THEN "Grocery/Discount/Club" WHEN industry_name = "Grocery" THEN "Grocery/Discount/Club" WHEN industry_name = "Health Products/Services" THEN "Grocery/Discount/Club" WHEN industry_name = "Discount/Club" THEN "Grocery/Discount/Club" WHEN industry_name = "Home & Garden" THEN "Hardlines" WHEN industry_name = "Automotive" THEN "Hardlines" WHEN industry_name = "Beauty" THEN "Hardlines" WHEN industry_name = "Sporting Goods, Hobby, Toy & Game" THEN "Hardlines" WHEN industry_name = "Commercial - Hardlines" THEN "Hardlines" WHEN industry_name = "Pet Products/Services" THEN "Hardlines" WHEN industry_name = "Limited-Service Restaurants" THEN "Restaurants" WHEN industry_name = "Full-Service Restaurants" THEN "Restaurants" WHEN industry_name = "Apparel, Accessories & Footwear" THEN "Softlines" WHEN industry_name = "Luxury" THEN "Softlines" WHEN industry_name = "Department Stores" THEN "Softlines" WHEN industry_name = "Electronics/Software" THEN "TMT+" WHEN industry_name = "Miscellaneous Services" THEN "TMT+" WHEN industry_name = "Online Retail - Broadlines" THEN "TMT+" WHEN industry_name = "Telecom & Media" THEN "TMT+" WHEN industry_name = "Education" THEN "TMT+" WHEN industry_name = "Other Specialty Retail" THEN "TMT+" WHEN industry_name = "Leisure & Recreation" THEN "Travel/Leisure" WHEN industry_name = "Travel" THEN "Travel/Leisure" WHEN industry_name = "Ground Transportation" THEN "Travel/Leisure" ELSE "" END as Insights_Group FROM(SELECT *, LAG(rolling_7d_spend_growth,364) over(order by symbol, segment_name, trans_date) as rolling_7d_spend_growth_ya, LAG(rolling_35d_spend_growth,364) over(order by symbol, segment_name, trans_date) as rolling_35d_spend_growth_ya FROM (SELECT *, ROUND((rolling_7d_spend/LAG(rolling_7d_spend,364) over(order by symbol, segment_name, brand_name, trans_date)-1),1) as rolling_7d_spend_growth, ROUND((rolling_35d_spend/LAG(rolling_35d_spend,364) over(order by symbol, segment_name, brand_name, trans_date, trans_date)-1),1) as rolling_35d_spend_growth FROM (SELECT t.symbol, t.segment_name, t.brand_name, t.trans_date, t.trans_count, t.spend_amount_usd, industry_name, ind.subindustry_name, SUM(spend_amount_usd) over(order by t.symbol, t.segment_name, t.brand_name, t.trans_date ROWS BETWEEN 6 PRECEDING and CURRENT Row) as rolling_7d_spend, SUM(t.spend_amount_usd) over(order by t.symbol, t.segment_name, t.brand_name, t.trans_date ROWS BETWEEN 34 PRECEDING and CURRENT Row) as rolling_35d_spend FROM `beta-omega-alpha-theta.apollo_us_cube_daily_signal.day_sym_seg_brand_emax` t INNER JOIN `beta-omega-alpha-theta.ground_truth.symbol_segment` sym_seg ON t.segment_name = sym_seg.segment_name INNER JOIN `beta-omega-alpha-theta.ground_truth.symbol_detail` sym_det ON sym_seg.symbol_id = sym_det.symbol_id INNER JOIN `beta-omega-alpha-theta.ground_truth.industry_list` ind ON sym_det.subindustry_id = ind.subindustry_id where seg_prod.primary_segment_flag = 1 and product_id = 1000 and t.segment_name NOT LIKE "%ACQ%" and t.segment_name NOT LIKE "%DLST%" and sym_det.ownership_status NOT LIKE "%Private%" order by symbol, t.segment_name, trans_date))) order by symbol, segment_name, trans_date DESC'
#us1_day_brand_df = client.query(us1_day_brand_query).to_dataframe()
#us1_day_brand_df.sort_values(by = ['symbol', 'segment_name', 'trans_date'], inplace = True)

us1_2_day_brand_query = 'SELECT *, CASE WHEN industry_name = "Financial Services" THEN "Financial Services"  WHEN industry_name = "Insurance" THEN "Financial Services" WHEN industry_name = "Convenience/Drug/Diet" THEN "Grocery/Discount/Club" WHEN industry_name = "Grocery" THEN "Grocery/Discount/Club" WHEN industry_name = "Health Products/Services" THEN "Grocery/Discount/Club" WHEN industry_name = "Discount/Club" THEN "Grocery/Discount/Club" WHEN industry_name = "Home & Garden" THEN "Hardlines" WHEN industry_name = "Automotive" THEN "Hardlines" WHEN industry_name = "Beauty" THEN "Hardlines" WHEN industry_name = "Sporting Goods, Hobby, Toy & Game" THEN "Hardlines" WHEN industry_name = "Commercial - Hardlines" THEN "Hardlines" WHEN industry_name = "Pet Products/Services" THEN "Hardlines" WHEN industry_name = "Limited-Service Restaurants" THEN "Restaurants" WHEN industry_name = "Full-Service Restaurants" THEN "Restaurants" WHEN industry_name = "Apparel, Accessories & Footwear" THEN "Softlines" WHEN industry_name = "Luxury" THEN "Softlines" WHEN industry_name = "Department Stores" THEN "Softlines" WHEN industry_name = "Electronics/Software" THEN "TMT+" WHEN industry_name = "Miscellaneous Services" THEN "TMT+" WHEN industry_name = "Online Retail - Broadlines" THEN "TMT+" WHEN industry_name = "Telecom & Media" THEN "TMT+" WHEN industry_name = "Education" THEN "TMT+" WHEN industry_name = "Other Specialty Retail" THEN "TMT+" WHEN industry_name = "Leisure & Recreation" THEN "Travel/Leisure" WHEN industry_name = "Travel" THEN "Travel/Leisure" WHEN industry_name = "Ground Transportation" THEN "Travel/Leisure" ELSE "" END as Insights_Group FROM(SELECT *, LAG(rolling_7d_spend_growth,364) over(order by symbol, segment_name, brand_name, trans_date) as rolling_7d_spend_growth_ya, LAG(rolling_35d_spend_growth,364) over(order by symbol, segment_name, brand_name, trans_date) as rolling_35d_spend_growth_ya FROM (SELECT *, ROUND((rolling_7d_spend/LAG(rolling_7d_spend,364) over(order by symbol, segment_name, brand_name, trans_date)-1),3) as rolling_7d_spend_growth, ROUND((rolling_35d_spend/LAG(rolling_35d_spend,364) over(order by symbol, segment_name, brand_name, trans_date, trans_date)-1),3) as rolling_35d_spend_growth FROM (SELECT t.symbol, t.segment_name, t.brand_name, t.trans_date, t.trans_count, t.spend_amount_usd, industry_name, ind.subindustry_name, SUM(spend_amount_usd) over(order by t.symbol, t.segment_name, t.brand_name, t.trans_date ROWS BETWEEN 6 PRECEDING and CURRENT Row) as rolling_7d_spend, SUM(t.spend_amount_usd) over(order by t.symbol, t.segment_name, t.brand_name, t.trans_date ROWS BETWEEN 34 PRECEDING and CURRENT Row) as rolling_35d_spend FROM `beta-omega-alpha-theta.combined_us1_us2_cube_daily_signal.day_sym_seg_brand_emax` t INNER JOIN `beta-omega-alpha-theta.ground_truth.symbol_segment` sym_seg ON t.segment_name = sym_seg.segment_name INNER JOIN `beta-omega-alpha-theta.ground_truth.symbol_detail` sym_det ON sym_seg.symbol_id = sym_det.symbol_id INNER JOIN `beta-omega-alpha-theta.ground_truth.industry_list` ind ON sym_det.subindustry_id = ind.subindustry_id LEFT JOIN `consumeredgeresearch-169820.ground_truth.segment_product` seg_prod ON sym_seg.segment_id = seg_prod.segment_id where seg_prod.primary_segment_flag = 1 and product_id = 1000 and t.segment_name NOT LIKE "%ACQ%" and t.segment_name NOT LIKE "%DLST%" and sym_det.ownership_status NOT LIKE "%Private%" order by symbol, t.segment_name, trans_date))) order by symbol, segment_name, trans_date DESC'
us1_2_day_brand_df = client.query(us1_2_day_brand_query).to_dataframe()
us1_2_day_brand_df.sort_values(by = ['symbol', 'segment_name', 'trans_date'], inplace = True)

C:\Users\Greg\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
C:\Users\Greg\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [6]:
#Date Transformations on USA1 segs
#date_transf(us1_day_seg_df)

In [7]:
#Date Transformations on USA2 segs
date_transf(us1_2_day_seg_df)

In [8]:
#Date Transformations on USA1 brands
#date_transf(us1_day_brand_df)

In [9]:
#Date Transformations on USA2 brands
date_transf(us1_2_day_brand_df)

In [10]:
#Fetch ground truth data: Industry/Subindustry/Groups/Primary Segments
symbol_segment_brand_industry_query = 'SELECT *, CASE WHEN industry_name = "Financial Services" THEN "Financial Services"  WHEN industry_name = "Insurance" THEN "Financial Services" WHEN industry_name = "Convenience/Drug/Diet" THEN "Grocery/Discount/Club" WHEN industry_name = "Grocery" THEN "Grocery/Discount/Club" WHEN industry_name = "Health Products/Services" THEN "Grocery/Discount/Club" WHEN industry_name = "Discount/Club" THEN "Grocery/Discount/Club" WHEN industry_name = "Home & Garden" THEN "Hardlines" WHEN industry_name = "Automotive" THEN "Hardlines" WHEN industry_name = "Beauty" THEN "Hardlines" WHEN industry_name = "Sporting Goods, Hobby, Toy & Game" THEN "Hardlines" WHEN industry_name = "Commercial - Hardlines" THEN "Hardlines" WHEN industry_name = "Pet Products/Services" THEN "Hardlines" WHEN industry_name = "Limited-Service Restaurants" THEN "Restaurants" WHEN industry_name = "Full-Service Restaurants" THEN "Restaurants" WHEN industry_name = "Apparel, Accessories & Footwear" THEN "Softlines" WHEN industry_name = "Luxury" THEN "Softlines" WHEN industry_name = "Department Stores" THEN "Softlines" WHEN industry_name = "Electronics/Software" THEN "TMT+" WHEN industry_name = "Miscellaneous Services" THEN "TMT+" WHEN industry_name = "Online Retail - Broadlines" THEN "TMT+" WHEN industry_name = "Telecom & Media" THEN "TMT+" WHEN industry_name = "Education" THEN "TMT+" WHEN industry_name = "Other Specialty Retail" THEN "TMT+" WHEN industry_name = "Leisure & Recreation" THEN "Travel/Leisure" WHEN industry_name = "Travel" THEN "Travel/Leisure" WHEN industry_name = "Ground Transportation" THEN "Travel/Leisure" ELSE "" END as insights_group FROM (SELECT distinct symbol, segment_name, brand_name, industry_name, subindustry_name FROM `consumeredgeresearch-169820.ground_truth.symbol_segment` sym_seg LEFT JOIN `consumeredgeresearch-169820.ground_truth.segment_brand` seg_brand ON sym_seg.segment_id = seg_brand.segment_id LEFT JOIN `consumeredgeresearch-169820.ground_truth.brand_detail` brand_det ON seg_brand.brand_id = brand_det.brand_id LEFT JOIN `consumeredgeresearch-169820.ground_truth.symbol_detail` sym_det ON sym_seg.symbol_id = sym_det.symbol_id LEFT JOIN `consumeredgeresearch-169820.ground_truth.segment_product` seg_prod ON sym_seg.segment_id = seg_prod.segment_id LEFT JOIN `consumeredgeresearch-169820.ground_truth.industry_list` ind ON sym_det.subindustry_id = ind.subindustry_id where product_id = 1000 and primary_segment_flag = 1 and publish_flag = 2 and symbol NOT LIKE "%ACQ%" and symbol NOT LIKE "%DLST%")'
symbol_segment_brand_industry_df = client.query(symbol_segment_brand_industry_query).to_dataframe()

#Filter data based on selected group
symbol_segment_brand_industry_df_filt = symbol_segment_brand_industry_df[symbol_segment_brand_industry_df['insights_group'] == group]
primary_segments = list(symbol_segment_brand_industry_df_filt['segment_name'].unique())
segment_brands = symbol_segment_brand_industry_df_filt[['segment_name', 'brand_name']]

C:\Users\Greg\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [11]:
selected_segment = 'DDS (US)'

single_plot_df = us1_2_day_seg_df[us1_2_day_seg_df['segment_name'] == selected_segment][(us1_2_day_seg_df['trans_date_dt'] >= first_trans_date)]

fig = px.line()

fig.add_scatter(x = single_plot_df['norm_date'], y = single_plot_df['rolling_7d_spend_growth'], name = str(selected_segment) + ' - Roll_7D')
fig.add_scatter(x = single_plot_df['norm_date'], y = single_plot_df['rolling_7d_spend_growth_ya'], name = str(selected_segment) + ' - Roll_7D_YA')
fig.add_scatter(x = single_plot_df['norm_date'], y = ((1+single_plot_df['rolling_7d_spend_growth_ya'])*(1+single_plot_df['rolling_7d_spend_growth']))-1, name = str(selected_segment) + ' - Roll_7D_2Y_Stack')

fig.add_scatter(x = single_plot_df['norm_date'], y = single_plot_df['rolling_35d_spend_growth'], name = str(selected_segment) + ' - Roll_35D')
fig.add_scatter(x = single_plot_df['norm_date'], y = single_plot_df['rolling_35d_spend_growth_ya'], name = str(selected_segment) + ' - Roll_35D_YA')
fig.add_scatter(x = single_plot_df['norm_date'], y = ((1+single_plot_df['rolling_35d_spend_growth_ya'])*(1+single_plot_df['rolling_35d_spend_growth']))-1, name = str(selected_segment) + ' - Roll_35D_2Y_Stack')


fig.update_yaxes(tickformat=".1%")

fig.update_xaxes(tickangle=45,
                 tickmode = 'array',
                 tickvals = single_plot_df['norm_date'][0::7])

fig.update_layout(title_text=selected_segment, title_x=0.45)

fig.show()

C:\Users\Greg\AppData\Local\Temp\ipykernel_7588\292061908.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  single_plot_df = us1_2_day_seg_df[us1_2_day_seg_df['segment_name'] == selected_segment][(us1_2_day_seg_df['trans_date_dt'] >= first_trans_date)]


In [12]:
#Filter on transaction date selected in 2nd cell
us1_2_day_seg_df_filt = us1_2_day_seg_df[['symbol', 'segment_name', 'norm_date', 'norm_date_ya','year','month_day', 'rolling_7d_spend_growth', 'rolling_7d_spend_growth_ya', 'rolling_35d_spend_growth', 'rolling_35d_spend_growth_ya']][(us1_2_day_seg_df['trans_date_dt'] >= first_trans_date)]
us1_2_day_brand_df_filt = us1_2_day_brand_df[['symbol', 'segment_name', 'brand_name', 'norm_date', 'norm_date_ya','year','month_day', 'rolling_7d_spend_growth', 'rolling_7d_spend_growth_ya', 'rolling_35d_spend_growth', 'rolling_35d_spend_growth_ya']][(us1_2_day_brand_df['trans_date_dt'] >= first_trans_date)]

In [13]:
def make_seg_chart(df, segment):

    fig = px.line()

    fig.add_scatter(x = df['norm_date'], y = df['rolling_7d_spend_growth'], name = str(segment) + ' - Roll_7D')
    fig.add_scatter(x = df['norm_date'], y = df['rolling_7d_spend_growth_ya'], name = str(segment) + ' - Roll_7D_YA')
    fig.add_scatter(x = df['norm_date'], y = ((1+df['rolling_7d_spend_growth_ya'])*(1+df['rolling_7d_spend_growth']))-1, name = str(segment) + ' - Roll_7D_2Y_Stack')
    fig.add_scatter(x = df['norm_date'], y = df['rolling_35d_spend_growth'], name = str(segment) + ' - Roll_35D', visible='legendonly')
    fig.add_scatter(x = df['norm_date'], y = df['rolling_35d_spend_growth_ya'], name = str(segment) + ' - Roll_35D_YA', visible='legendonly')
    fig.add_scatter(x = df['norm_date'], y = ((1+df['rolling_35d_spend_growth_ya'])*(1+df['rolling_35d_spend_growth']))-1, name = str(segment) + ' - Roll_35D_2Y_Stack', visible='legendonly')

    fig.update_yaxes(tickformat=".1%")

    fig.update_xaxes(tickangle=45,
                 tickmode = 'array',
                 tickvals = single_plot_df['norm_date'][0::5])

    fig.update_layout(title_text=str(segment), title_x=0.45)
    fig.show()
    #fig.write_image(filepath)
    
def make_brand_chart(df, segment, brand, index):

    fig = px.line()
    fig.add_scatter(x = df['norm_date'], y = df['rolling_7d_spend_growth'], name = str(brand) + ' - Roll_7D')
    fig.add_scatter(x = df['norm_date'], y = df['rolling_7d_spend_growth_ya'], name = str(brand) + ' - Roll_7D_YA')

    fig.update_yaxes(tickformat=".1%")

    fig.update_xaxes(tickangle=45,
                 tickmode = 'array',
                 tickvals = single_plot_df['norm_date'][0::5])

    fig.update_layout(title_text=str(segment)+': '+str(brand), title_x=0.45)
    
    #fig.write_image(filepath)


In [14]:
for i, seg in enumerate(primary_segments):

    segment = primary_segments[i]
    segment_brand = list(segment_brands[segment_brands['segment_name']==segment]['brand_name'])

    plot_seg_df = us1_2_day_seg_df_filt[['norm_date','norm_date_ya', 'year', 'month_day', 'rolling_7d_spend_growth', 'rolling_7d_spend_growth_ya', 'rolling_35d_spend_growth', 'rolling_35d_spend_growth_ya']][us1_2_day_seg_df_filt['segment_name']==segment]

    if ':' in segment:
        segment = segment.replace(":","")
        img_name = segment + ' 7D Rolling Avg.png'
    else:
        img_name = segment + ' 7D Rolling Avg.png'
    
    #filepath = Path(root+date_path+img_name)

    make_seg_chart(plot_seg_df, segment)
    #make_seg_chart(plot_seg_df, segment, filepath)


    '''for x in range(0,len(segment_brand)):

        brand = segment_brand[x]

        plot_brand_df = us1_2_day_brand_df_filt[['norm_date','norm_date_ya', 'year', 'month_day', 'rolling_7d_spend_growth', 'rolling_7d_spend_growth_ya', 'rolling_35d_spend_growth', 'rolling_35d_spend_growth_ya']][us1_2_day_brand_df_filt['brand_name']==segment_brand[x]]
        
        img_name = brand + ' 7D Rolling Avg.png'
        filepath = Path(root+date_path+img_name)
        
        make_brand_chart(plot_brand_df, segment, segment_brand[x], x, filepath)
        #make_brand_chart(plot_brand_df, segment, segment_brand[x], x, filepath)'''


        

In [15]:
images = [
    Image.open("C:/Users/test/Desktop/testpics/" + f)
    for f in ["pic1.jpg", "pic2.jpg", "pic3.jpg"]
]

pdf_path = "C:/Users/test/Desktop/pics123.pdf"
    
images[0].save(
    pdf_path, "PDF" ,resolution=100.0, save_all=True, append_images=images[1:]

SyntaxError: incomplete input (743756857.py, line 9)